In [1]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from datetime import datetime, timedelta
import time
import os
import pandas as pd

all_news_links = []
    
#RESTART KERNEL IF SCRIPT DOESN'T WORK THE SECOND TIME

BASE_URL = "https://telanganatoday.com/"
country = 'bangladesh'
initial_url = f'{BASE_URL}search?q={country}'

# chrome_options = Options()
# chrome_options.add_argument("--headless")
# driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()
driver.get(initial_url)

#this Website displays news articles with the help of Google/enchanced by Google.
#So, each page needs to be separately selected and visited to scrape the links. 
#that can't be done by changing the Page Number in the URl. So, it's done using ActionsChains 'Click' function.

page = 1
while page < 3:
    
    try:
        buttons = driver.find_elements(By.CLASS_NAME, 'gsc-cursor-page')
#         print(buttons)
        
        for button in buttons: 
                
            if button.get_attribute('innerHTML') == f'{page}':

                ActionChains(driver)\
                .click(button)\
                .perform()
                time.sleep(15)                  

                main_div = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div[2]/div/div/div/div[5]/div[2]/div/div')
#                 print(main_div)
                print(f'in page {page}')
                      
                if main_div:
                    
                    news = main_div.find_elements(By.CSS_SELECTOR, 'a')
                    print('found main_div')
                      
                    for n in news:
                        news_text = n.get_attribute('data-ctorig')
                      
                        print('Appending News Links')
                      
                        if news_text in all_news_links:
                            continue
                        else:
                            all_news_links.append(news_text)
                            
                page+=1
                              
    except:
       
            pass

driver.quit()

#For some reason, a None object gets appended to the list. So, removing that here.
modified_news_links = [i for i in all_news_links if i is not None]

#SCRAPING THE LINKS

counter = 0
data_list = []

for url in modified_news_links:

        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE

        title_tag = soup.find('h1', class_ = 'articleHD')
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #Content Summary
        content_summary_tag = soup.find('p', class_ = 'excerpt')
        content_summary = content_summary_tag.text if content_summary_tag else 'Content Summary not found'
        summary_translation = 'None'

        #AUTHOR
        author_div = soup.find('div', class_ = 'author-name-wrap')
        author = author_div.find('a').text if author_div else 'Author not found'

        #Date 

        date_info = soup.find('meta', {'property':'article:modified_date'})

        if date_info:
            
                date_data = date_info.get('content')
                only_date = date_data.split('T')[0]
                only_time = date_data.split('T')[1]
                time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
                cleaned_date = f"{only_date},{time}"

                source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M")
                bangladesh_localtime = source_localtime + timedelta(minutes=30)

        else:

                date_data = 'Date data not found'


        #Content

        content = []

        content_div = soup.find('div', class_ = 'detailBody')

        if content_div:

            all_paras = content_div.find_all('p')

            relevant_paras = all_paras[1:]

            for each_para in relevant_paras:

                content.append(each_para.text)

            full_content= ''.join(content)
            full_content = re.sub('\n|\r',' ',full_content)
            full_content = full_content.replace('\xa0','')
            full_content = full_content.strip()

        else:

            full_content = 'Content not found'

        content_translation = 'None'
        
        data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')


                
df = pd.DataFrame(data_list)
df.head()

csv_filename = f"{country}_Telangana_Today.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)


in page 1
found main_div
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
in page 2
found main_div
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appending News Links
Appen

,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://telanganatoday.com/india-bangladesh-to...,"India, Bangladesh to secure ‘chicken’s neck’ w...",New Delhi: In a decision that will have huge g...,Prime Minister Narendra Modi and his Banglades...,None,None,None,IANS,bangladesh,2020-12-14 16:52:00,2020-12-14 17:22:00
1,https://telanganatoday.com/cwc-2023-bangladesh...,CWC 2023: Bangladesh aim to revive campaign ag...,"Kolkata: Seeking a revival of fortunes, Bangla...",Both teams have one win from four outings. Cal...,None,None,None,PTI,bangladesh,2023-10-27 23:02:00,2023-10-27 23:32:00
2,https://telanganatoday.com/bangladesh-17-kille...,Bangladesh: 17 killed in road accident in Jhal...,Dhaka: At least 17 people were killed and more...,The accident took place in the Jhalakathi dist...,None,None,None,PTI,bangladesh,2023-07-22 22:12:00,2023-07-22 22:42:00
3,https://telanganatoday.com/tamim-iqbal-reverse...,Tamim Iqbal reverses retirement decision after...,Dhaka: A day after announcing his shocking ret...,The 34-year old cricketer met the Prime Minist...,None,None,None,IANS,bangladesh,2023-07-07 20:05:00,2023-07-07 20:35:00
4,https://telanganatoday.com/protest-rallies-hel...,"Protest rallies held in Bangladesh, demand jus...",Dhaka: Amid the world’s criticism of the atroc...,Amid the world's criticism of the atrocities d...,None,None,None,ANI,bangladesh,2023-10-02 09:26:00,2023-10-02 09:56:00


In [2]:
df.head()

,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://telanganatoday.com/india-bangladesh-to...,"India, Bangladesh to secure ‘chicken’s neck’ w...",New Delhi: In a decision that will have huge g...,Prime Minister Narendra Modi and his Banglades...,None,None,None,IANS,bangladesh,2020-12-14 16:52:00,2020-12-14 17:22:00
1,https://telanganatoday.com/cwc-2023-bangladesh...,CWC 2023: Bangladesh aim to revive campaign ag...,"Kolkata: Seeking a revival of fortunes, Bangla...",Both teams have one win from four outings. Cal...,None,None,None,PTI,bangladesh,2023-10-27 23:02:00,2023-10-27 23:32:00
2,https://telanganatoday.com/bangladesh-17-kille...,Bangladesh: 17 killed in road accident in Jhal...,Dhaka: At least 17 people were killed and more...,The accident took place in the Jhalakathi dist...,None,None,None,PTI,bangladesh,2023-07-22 22:12:00,2023-07-22 22:42:00
3,https://telanganatoday.com/tamim-iqbal-reverse...,Tamim Iqbal reverses retirement decision after...,Dhaka: A day after announcing his shocking ret...,The 34-year old cricketer met the Prime Minist...,None,None,None,IANS,bangladesh,2023-07-07 20:05:00,2023-07-07 20:35:00
4,https://telanganatoday.com/protest-rallies-hel...,"Protest rallies held in Bangladesh, demand jus...",Dhaka: Amid the world’s criticism of the atroc...,Amid the world's criticism of the atrocities d...,None,None,None,ANI,bangladesh,2023-10-02 09:26:00,2023-10-02 09:56:00
